# Clustering Mapper

## Importation des modules

In [141]:
# Modules de base
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Clustering
from sklearn.cluster import AgglomerativeClustering 
# model = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='single')
# model.fit(X)
# labels = model.labels_

# Mapper
import kmapper as km
from kmapper.cover import Cover
from kmapper import jupyter


## Chargement des données

In [142]:
data_firm_level = pd.read_stata("../data/Firm_patent/data_firm_level.dta")
# data_patent_level = pd.read_stata("../data/Patent_level_data/data_patent_level.dta")
# cites = pd.read_stata("../data/Patent_level_data/USPatent_1926-2010/cites/cites.dta")
# firm_innovation_v2 = pd.read_stata("../data/Patent_level_data/USPatent_1926-2010/firm_innovation/firm_innovation_v2.dta)
# patents_xi = pd.read_stata("../data/Patent_level_data/USPatent_1926-2010/patents_xi/patents_xi.dt)
# patent_values = pd.read_stata("../data/Patent_level_data/Patent_CRSP_match_1929-2017/patent_values/patent_values.dta")
patents_firm_merge = pd.read_stata("../data/Firm_patent/patents_firm_merge.dta")

## Preprocessing

### On garde les grandes entreprises

In [206]:
pp_df = patents_firm_merge.copy()

pp_df["patent_class_int"] = pd.to_numeric(pp_df["patent_class"], errors="coerce")

pp_df = pp_df.dropna(subset=["patent_class_int", "xi", "tsm", "Tsm", "tcw", "Tcw"])

pp_df["patent_class_int"] = pp_df["patent_class_int"].astype(int)

for col in ["fdate", "idate", "pdate"]:
    a = pd.to_datetime(pp_df[col], format="%m/%d/%Y", errors="coerce")
    pp_df[col] = a

pp_df.set_index("patnum", inplace=True)

In [207]:
print("len(patents_firm_merge):", len(patents_firm_merge))
print("len(pp_df):", len(pp_df), "\n")
print("pp_df columns %:")
print(pp_df.count()/len(pp_df))

len(patents_firm_merge): 1844881
len(pp_df): 1635631 

pp_df columns %:
index               1.000000
fdate               0.994111
idate               1.000000
pdate               0.252904
permno              1.000000
patent_class        1.000000
subclass            1.000000
ncites              1.000000
xi                  1.000000
year                1.000000
Npats               1.000000
Tcw                 1.000000
Tsm                 1.000000
tcw                 1.000000
tsm                 1.000000
_merge              1.000000
patent_class_int    1.000000
dtype: float64


In [208]:
big_firms_data = pp_df[pp_df["permno"].isin(pp_df["permno"].value_counts().index[:5])]

In [209]:
main_df = big_firms_data[["idate", "year", "permno", "xi", "patent_class_int"]]

In [210]:
main_df

,idate,year,permno,xi,patent_class_int
patnum,,,,,
2510623,1950-06-06,1950,12060.0,0.321411,327
2509729,1950-05-30,1950,12060.0,0.376306,348
2494352,1950-01-10,1950,12060.0,0.221822,368
2497686,1950-02-14,1950,12060.0,0.128556,74
2511666,1950-06-13,1950,12060.0,0.280345,96
...,...,...,...,...,...
7786437,2010-08-31,2010,64231.0,0.067182,250
7809502,2010-10-05,2010,64231.0,0.059250,701
7661187,2010-02-16,2010,64231.0,0.033786,29


In [211]:
final_df = pd.get_dummies(data=main_df, columns=["patent_class_int"], prefix=["pc"]) #, sparse=True)

In [212]:
final_df

,idate,year,permno,xi,pc_1,pc_2,pc_4,pc_5,pc_7,pc_8,...,pc_715,pc_716,pc_717,pc_718,pc_719,pc_720,pc_725,pc_726,pc_800,pc_850
patnum,,,,,,,,,,,,,,,,,,,,,
2510623,1950-06-06,1950,12060.0,0.321411,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2509729,1950-05-30,1950,12060.0,0.376306,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2494352,1950-01-10,1950,12060.0,0.221822,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2497686,1950-02-14,1950,12060.0,0.128556,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2511666,1950-06-13,1950,12060.0,0.280345,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7786437,2010-08-31,2010,64231.0,0.067182,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7809502,2010-10-05,2010,64231.0,0.059250,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7661187,2010-02-16,2010,64231.0,0.033786,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [272]:
firm_df = final_df.groupby(by=["permno", "year"]).agg({s : 'sum' for s in final_df.columns if s[:2]=="pc"}).reset_index()

In [360]:
firm_df.to_pickle("../data/tables/firm_df.pkl")

### On lisse les données par rapport au temps

In [307]:
features = [s for s in final_df.columns if s[:2]=="pc"]
SMA_features = ["SMA_"+l for l in features]
log_features = ["log_"+l for l in features]

In [297]:
SMA_df = firm_df.groupby(["permno"]
)[features
].rolling(window=5, min_periods=5 # =5?
).sum(
).rename(columns={l: "SMA_"+l for l in features}
).reset_index()

In [342]:
SMA_df["year"] = firm_df["year"]
SMA_df.drop("level_1", axis=1, inplace=True)
SMA_df.dropna(inplace=True)

In [359]:
SMA_df.to_pickle("../data/tables/SMA_df.pkl")

In [344]:
SMA_dist = SMA_df.set_index(["permno", "year"]).transpose().corr()

In [357]:
SMA_dist.to_pickle("../data/tables/SMA_dist.pkl")

In [308]:
log_df = np.log(1 + SMA_df[SMA_features]).rename(columns={"SMA_"+l: "log_"+l for l in features})
log_df["permno"] = SMA_df["permno"]
log_df["year"] = SMA_df["year"]
log_df.dropna(inplace=True)

In [358]:
log_df.to_pickle("../data/tables/log_df.pkl")

In [325]:
log_dist = log_df.set_index(["permno", "year"]).transpose().corr()

In [354]:
log_dist.to_pickle("../data/tables/log_dist.pkl")

In [373]:
matrix = log_df.set_index(["permno", "year"])

In [306]:
data_firm_level

,year,gvkey,fyr,csho,ebit,ebitda,emp,pstk,sale,xrd,...,rivao,rintan,ract,value,value_e,value_d,qkstock,tobinq,tobinq_e,tobinq_d
0,1994.0,10846,12.0,363.661,3856.000,4977.000,304.000,NaN,45419.000,831.000,...,148.300720,0.000000,15795.056641,NaN,NaN,-10743.563477,19477.857422,NaN,NaN,-0.551578
1,1995.0,10846,12.0,363.800,3986.000,5242.000,308.000,NaN,49732.000,923.000,...,169.000000,0.000000,16314.000000,NaN,NaN,-11067.000000,20419.000000,NaN,NaN,-0.541995
2,1996.0,10846,12.0,363.816,4473.000,5794.000,306.000,153.435,52161.000,934.000,...,187.683273,0.000000,16892.472656,51197.640625,62475.253906,-11277.615234,19883.673828,2.574858,3.142038,-0.567180
3,1996.0,153351,12.0,15.669,-3.810,-3.766,0.012,0.001,2.000,3.344,...,0.000000,0.000000,47.806450,124.507561,172.314011,-47.806450,0.085044,1464.037354,2026.175293,-562.137939
4,1996.0,27961,12.0,NaN,15.514,20.698,0.915,0.000,170.821,0.900,...,0.000000,144.434006,57.626587,NaN,NaN,98.776138,204.719437,NaN,NaN,0.482495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12982,2010.0,160329,12.0,321.301,10381.000,11762.000,24.400,0.000,29321.000,3762.000,...,364.714081,5090.655762,28983.263672,106517.539062,133084.484375,-26566.945312,10866.109375,9.802731,12.247666,-2.444936
12983,2010.0,9323,12.0,1.554,-1.242,-1.166,0.036,0.000,8.036,0.201,...,0.000000,0.000000,5.190377,-2.318619,2.871758,-5.190377,0.533473,-4.346274,5.383137,-9.729411
12984,2010.0,61214,12.0,436.593,1659.553,1858.611,9.245,0.000,5981.583,694.534,...,26.746164,143.364716,4575.182129,7756.094727,11672.926758,-3916.832031,2245.137451,3.454619,5.199203,-1.744585
12985,2010.0,6100,12.0,45.032,235.873,257.998,0.300,0.000,394.545,71.464,...,1.185495,90.868202,432.047424,875.902710,1307.623779,-431.721069,97.872383,8.949437,13.360498,-4.411061


### On applique le Mapper Algorithm

In [369]:
# matrix.reset_index(drop=True, inplace=True)
matrix

,log_pc_1,log_pc_2,log_pc_4,log_pc_5,log_pc_7,log_pc_8,log_pc_14,log_pc_15,log_pc_16,log_pc_19,...,log_pc_715,log_pc_716,log_pc_717,log_pc_718,log_pc_719,log_pc_720,log_pc_725,log_pc_726,log_pc_800,log_pc_850
0,0.000000,0.000000,1.386294,0.693147,0.0,0.000000,0.0,1.791759,1.609438,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147
1,0.000000,0.000000,1.386294,0.693147,0.0,0.000000,0.0,1.791759,1.791759,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147
2,0.000000,0.000000,1.386294,0.000000,0.0,0.000000,0.0,1.945910,1.791759,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147
3,0.000000,0.000000,1.386294,0.693147,0.0,0.000000,0.0,1.791759,1.791759,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.693147,0.693147,0.0,0.000000,0.0,1.609438,1.386294,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,5.159055,0.693147,0.000000,0.000000,0.0,0.000000,0.0,1.386294,0.000000,0.0,...,3.433987,4.189655,3.332205,2.944439,2.197225,3.555348,2.397895,2.833213,0.693147,4.043051
208,5.283204,0.693147,0.000000,0.693147,0.0,0.000000,0.0,1.609438,0.693147,0.0,...,3.555348,4.219508,3.465736,3.135494,2.197225,3.433987,2.397895,3.295837,1.098612,3.988984
209,5.308268,0.693147,0.000000,0.693147,0.0,0.000000,0.0,1.609438,1.098612,0.0,...,3.367296,3.912023,3.526361,3.178054,2.397895,3.496508,2.708050,3.688879,1.609438,3.737670
210,5.384495,0.693147,0.000000,0.693147,0.0,0.693147,0.0,1.609438,1.098612,0.0,...,3.465736,3.912023,3.583519,3.178054,2.397895,3.737670,2.772589,3.850148,1.945910,3.258097


In [371]:
# Initialize
mapper = km.KeplerMapper(verbose=1)

KeplerMapper(verbose=1)


In [399]:
cover = Cover(n_cubes=20, perc_overlap=0.5)
scaler = StandardScaler().fit(matrix)
projector = PCA(n_components=2)
clusterer = AgglomerativeClustering(n_clusters=5, linkage="single", affinity="precomputed").fit(log_dist)

In [394]:
log_dist

permno         12060.0                                                    \
year              1954      1955      1956      1957      1958      1959   
permno  year                                                               
12060.0 1954  1.000000  0.985041  0.961151  0.925507  0.914745  0.906296   
        1955  0.985041  1.000000  0.976318  0.943897  0.932221  0.919487   
        1956  0.961151  0.976318  1.000000  0.971619  0.953570  0.940326   
        1957  0.925507  0.943897  0.971619  1.000000  0.977532  0.959633   
        1958  0.914745  0.932221  0.953570  0.977532  1.000000  0.982725   
...                ...       ...       ...       ...       ...       ...   
64231.0 2006  0.345160  0.349485  0.358980  0.361531  0.377090  0.396668   
        2007  0.343575  0.348420  0.358349  0.363212  0.375240  0.396520   
        2008  0.336793  0.339549  0.350290  0.353721  0.362674  0.381529   
        2009  0.326613  0.328487  0.341915  0.346471  0.355659  0.374740   
        2010  0.325865  0.326587  0.343712  0.351287  0.358547  0.377754   

permno                                                ...   64231.0            \
year              1960      1961      1962      1963  ...      2001      2002   
permno  year                                          ...                       
12060.0 1954  0.892454  0.889724  0.890170  0.869615  ...  0.309941  0.311290   
        1955  0.895725  0.894067  0.894136  0.875172  ...  0.311244  0.311855   
        1956  0.915859  0.902780  0.897707  0.883034  ...  0.321716  0.320892   
        1957  0.935579  0.918262  0.895747  0.884395  ...  0.336163  0.334570   
        1958  0.958692  0.940586  0.919532  0.888770  ...  0.347161  0.349553   
...                ...       ...       ...       ...  ...       ...       ...   
64231.0 2006  0.409529  0.417387  0.432196  0.437986  ...  0.913872  0.928387   
        2007  0.407132  0.413747  0.427972  0.434513  ...  0.903865  0.908696   
        2008  0.393554  0.398389  0.412909  0.420067  ...  0.895884  0.900148   
        2009  0.386839  0.390844  0.403933  0.411270  ...  0.888311  0.891482   
        2010  0.389294  0.390158  0.402892  0.411433  ...  0.880787  0.881739   

permno                                                                    \
year              2003      2004      2005      2006      2007      2008   
permno  year                                                               
12060.0 1954  0.325901  0.331546  0.347573  0.345160  0.343575  0.336793   
        1955  0.328612  0.335643  0.352081  0.349485  0.348420  0.339549   
        1956  0.337981  0.343977  0.359495  0.358980  0.358349  0.350290   
        1957  0.350113  0.353469  0.361925  0.361531  0.363212  0.353721   
        1958  0.366779  0.371662  0.380273  0.377090  0.375240  0.362674   
...                ...       ...       ...       ...       ...       ...   
64231.0 2006  0.944621  0.957408  0.986786  1.000000  0.987838  0.976323   
        2007  0.923313  0.937387  0.969795  0.987838  1.000000  0.992033   
        2008  0.905159  0.919854  0.955258  0.976323  0.992033  1.000000   
        2009  0.894229  0.903177  0.941167  0.964847  0.983760  0.993453   
        2010  0.883052  0.891308  0.920332  0.945280  0.968172  0.980006   

permno                            
year              2009      2010  
permno  year                      
12060.0 1954  0.326613  0.325865  
        1955  0.328487  0.326587  
        1956  0.341915  0.343712  
        1957  0.346471  0.351287  
        1958  0.355659  0.358547  
...                ...       ...  
64231.0 2006  0.964847  0.945280  
        2007  0.983760  0.968172  
        2008  0.993453  0.980006  
        2009  1.000000  0.989528  
        2010  0.989528  1.000000  

[212 rows x 212 columns]

In [407]:
proj_matrix = mapper.fit_transform(X=matrix, projection=PCA(n_components=2), scaler=StandardScaler()) #, distance_matrix=log_dist)

..Composing projection pipeline of length 1:
	Projections: PCA(n_components=2)
	Distance matrices: False
	Scalers: StandardScaler()
..Projecting on data shaped (212, 398)

..Projecting data using: 
	PCA(n_components=2)


..Scaling with: StandardScaler()



In [409]:
# Create dictionary called 'graph' with nodes, edges and meta-information
graph = mapper.map(lens=proj_matrix, X=matrix, cover=Cover(n_cubes=20, perc_overlap=0.5), clusterer=AgglomerativeClustering(n_clusters=5, linkage="single"))

Mapping on data shaped (212, 398) using lens shaped (212, 2)

Creating 400 hypercubes.

Created 406 edges and 320 nodes in 0:00:00.139937.


In [410]:
# Visualize it
html = mapper.visualize(graph, path_html="../docs/mapper.html", title="Mapper Clustering Algorithm")

# Inline display
# jupyter.display(path_html="../docs/MapperCluster.html")

Wrote visualization to: ../docs/mapper.html
